<a href="https://colab.research.google.com/github/Marcosrbsantos/agente-posts-ai/blob/main/notebooks/agente_design_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instala Git e clona o repositório
!apt-get update -qq && apt-get install -qq git
!git clone https://github.com/Marcosrbsantos/agente-posts-ai.git
%cd agente-posts-ai

# 2. Instala bibliotecas gratuitas
!pip install --upgrade pip
!pip install diffusers transformers accelerate faiss-cpu sentence-transformers langchain bitsandbytes gradio


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Guia Técnico de Design Gráfico para Posts em Mídias Sociais.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print("Chunks criados:", len(chunks))


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS

embed_model = SentenceTransformer('all-MiniLM-L6-v2')
vectorstore = FAISS.from_documents(chunks, embed_model)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Carrega Falcon-7B-Instruct em 8-bit
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
model     = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b-instruct",
    device_map="auto",
    load_in_8bit=True
)
hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm     = HuggingFacePipeline(pipeline=hf_pipe)

# Monta o chain RAG
retriever = vectorstore.as_retriever(search_kwargs={"k":3})
qa        = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [ ]:
pergunta = "Como organizar camadas para um post de Instagram?"
resposta = qa.run(pergunta)
print("Pergunta:", pergunta)
print("Resposta:", resposta)


In [ ]:
import gradio as gr

def gerar_design(briefing: str):
    # 1) extrai spec via RAG
    spec = {
        "layers": qa.run("Explique a ordem de camadas para um post."),
        "colors": qa.run("Quais esquemas de cores complementares?"),
        "typography": qa.run("Quais regras tipográficas usar?")
    }
    # 2) monta prompt de difusão (exemplo simples)
    prompt = f"{spec['layers']} {spec['colors']} {spec['typography']} {briefing}"
    # 3) gera imagem
    from diffusers import StableDiffusionPipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype="auto"
    ).to("cuda")
    img = pipe(prompt, num_inference_steps=25, guidance_scale=7.5).images[0]
    return img

gr.Interface(
    fn=gerar_design,
    inputs="text",
    outputs="image",
    title="Agente de Design"
).launch()
